In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from itertools import cycle
colors = cycle(sns.color_palette())

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [3]:
from pearce.emulator import OriginalRecipe, ExtraCrispy, parameter, DEFAULT_PARAMS as PARAMS

In [4]:
training_dir = '/u/ki/swmclau2/des/PearceLHC_wp_rp'
fiducial_params = {'logM0': 12.20, 'logM1': 13.7, 'alpha': 1.02,
                      'logMmin': 12.1, 'f_c': 0.19, 'sigma_logM': 0.46}
rbins = np.array([  0.06309573,   0.12437607,   0.24517359,   0.34422476, 0.48329302, 0.67854546,\
           0.9526807 , 1.33756775,1.8779508 ,   2.6366509 ,   3.70186906,   5.19743987, 7.29722764, \
           10.24533859,  14.38449888,  20.1958975 , 28.35512583,  39.81071706] )
rpoints = (rbins[1:]+rbins[:-1])/2

In [5]:
y_param = 'logMmin'
emulation_point = [('f_c', 0.233), ('logM0', 12.0), ('sigma_logM', 0.533), 
                   ('alpha', 1.083),('logM1', 13.5), ('logMmin', 12.233)]
em_params = dict(emulation_point)
del em_params[y_param]

In [6]:
y_param_idx = -1
for idx, p in enumerate(PARAMS):
    if p.name==y_param:
        y_param_idx = idx
        break
        
y_vals = np.linspace(PARAMS[y_param_idx].low, PARAMS[y_param_idx].high, 4)
em_params[y_param] = y_vals

In [7]:
print em_params[y_param]

[ 11.7         11.96666667  12.23333333  12.5       ]


In [ ]:
print len(em_params)

6


In [ ]:
emu1 = ExtraCrispy(training_dir)

In [ ]:
#rpoints = np.logspace(np.log10(rbins[0]), np.log10(rbins[-1]), 200)
log_xi_pred, log_xi_cov = emu1.emulate_wrt_r(em_params, rpoints)
log_xi_err = np.sqrt(np.diag(log_xi_cov))
log_xi_pred = log_xi_pred.reshape((-1, len(rpoints)))
log_xi_err = log_xi_err.reshape((-1, len(rpoints)))

In [ ]:
truth_dir = '/u/ki/swmclau2/des/PearceFHC_wp_rp/'
plot_r, plot_xi, plot_xi_err = emu1.get_plot_data(em_params,truth_dir, independent_variable=None)

In [ ]:
plot_xi.shape, plot_r.shape, plot_xi_err.shape

In [ ]:
fig = plt.figure(figsize = (10, 10))
for label, lxp, lxe,pr, px, err,c in zip(y_vals, log_xi_pred, log_xi_err ,plot_r, plot_xi,plot_xi_err,colors):

    plt.plot(rpoints, lxp,label = '%.3f'%label, color = c )

    plt.fill_between(rpoints,lxp+lxe,
                     lxp-lxe, color = c, alpha = 0.3)

    plt.errorbar(10**pr, px,err, color =c,fmt = 'o')

plt.xlim(xmin = 0.1, xmax = 30)    
plt.ylim(ymin = -2, ymax = 4)
plt.title("OR Emulator w.r.t. %s"%y_param, fontsize =20)
#plt.loglog()
plt.xscale('log')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel(r'$r $  $\rm{[Mpc]}$', fontsize=25)
plt.ylabel(r'$\log_{10}{\wp_{\rm gg}(r_p) }$', fontsize=25)
plt.legend(loc='best', fontsize=25)

In [ ]:
#em_params[y_param] = em_params[y_param][1]
#rpoints = rpoints[:2]

In [ ]:
log_xi_pred_jk, log_xi_cov_jk = emu1.emulate_wrt_r(em_params, rpoints, jackknife_errors=True)

In [ ]:
log_xi_err_jk = np.sqrt(np.diag(log_xi_cov_jk))
log_xi_pred_jk = log_xi_pred_jk.reshape((-1, len(rpoints)))
log_xi_err_jk = log_xi_err_jk.reshape((-1, len(rpoints)))

In [ ]:
fig = plt.figure(figsize = (10, 10))
for label, lxp, lxe,pr, px, err,c in zip(y_vals, log_xi_pred_jk, log_xi_err_jk ,plot_r, plot_xi,plot_xi_err,colors):

    plt.plot(rpoints, lxp,label = '%.3f'%label, color = c )

    plt.fill_between(rpoints,lxp+lxe,
                     lxp-lxe, color = c, alpha = 0.3)

    plt.errorbar(10**pr, px,err, color =c,fmt = 'o')

plt.xlim(xmin = 0.1, xmax = 30)    
plt.ylim(ymin = -2, ymax = 4)
plt.title("OR Emulator w.r.t. %s"%y_param, fontsize =20)
#plt.loglog()
plt.xscale('log')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel(r'$r $  $\rm{[Mpc]}$', fontsize=25)
plt.ylabel(r'$\log_{10}{\wp_{\rm gg}(r_p) }$', fontsize=25)
plt.legend(loc='best', fontsize=25)

In [ ]:
for err, jk_err in zip(log_xi_err, log_xi_err_jk):
    plt.plot(jk_err/err);

In [ ]:
(log_xi_pred-plot_xi)/plot_xi

In [ ]:
rpoints

In [ ]:
abs_err = emu1.goodness_of_fit(truth_dir,2000, 'abs')

In [ ]:
rel_err = emu1.goodness_of_fit(truth_dir,2000, 'rel')

In [ ]:
abs_err.shape,rel_err.shape

In [ ]:
colors = sns.color_palette()

In [ ]:
fig = plt.figure(figsize = (20, 10))
plt.subplot(121)
for abe in abs_err:
    plt.plot(10**plot_r[0,:], abe, color = colors[0], alpha = 0.3)
plt.plot(10**plot_r[0,:], np.mean(abs_err, axis=0), color=colors[2])
plt.xlim(xmin = 0.1, xmax = 30)    
plt.ylim(ymin = -.1, ymax = .1)
plt.title("Abs err wprp", fontsize =20)
#plt.loglog()
plt.xscale('log')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel(r'$r $  $\rm{[Mpc]}$', fontsize=25)
plt.ylabel(r'$\log_{10}{wp_{\rm gg}(r_p) }$', fontsize=25)

plt.subplot(122)
for ree in rel_err:
    plt.plot(10**plot_r[0,:], ree,color = colors[0], alpha = 0.3)
plt.plot(10**plot_r[0,:], np.mean(rel_err, axis=0), color=colors[2])
plt.xlim(xmin = 0.1, xmax = 30)
plt.ylim(ymin = -.05, ymax = 0.05)
plt.title("Rel err wprp", fontsize =20)
plt.xscale('log')
#plt.loglog()
plt.xlabel(r'$r $  $\rm{[Mpc]}$', fontsize=25)

plt.show()

In [ ]:
r2 = emu1.goodness_of_fit(truth_dir,2000, 'r2')
r2.mean()